In [1]:
from datetime import date
from highlight_text import fig_text
from pathlib import Path

import os
import pandas as pd
import matplotlib.pyplot as plt
import bigframes.pandas as bpd

import warnings
warnings.filterwarnings(action='once')

%load_ext google.cloud.bigquery
bpd.options.bigquery.project = "dhh-ncr-stg"

/Users/patrick.doupe/DH/ad_hoc/cohorts/.venv/lib/python3.13/site-packages/google/cloud/bigquery/__init__.py:237: FutureWarning: %load_ext google.cloud.bigquery is deprecated. Install bigquery-magics package and use `%load_ext bigquery_magics`, instead.
  warnings.warn(


## Open questions

1. how can we flag vendor and chain?
2. how can we get hex data, or something nice and local
3. any other data that we're missing?


TODO
- want cross validation through time
- 


In [2]:
# Get the current script's directory
CURRENT_DIR = Path(os.getcwd())
# Navigate to parallel directory and read the SQL file
BASE_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "data_creation.sql"
CURRENT_COHORT_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "current_cohorts.sql"
CURRENT_RECOMMENDATIONS_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "current_recommendations.sql"
CURRENT_PERFORMANCE_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "current_performance.sql"
BEST_MATCH_RECOMMENDATIONS_SQL_FILE_PATH = CURRENT_DIR.parent / "queries" / "best_match_recommendations.sql"

In [19]:
with open(BASE_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT = file.read()

In [20]:
%%bigquery
$SQL_CONTENT

Executing query with job ID: 9eb95562-ce4d-44bc-8865-f8fc19765347
Query executing: 0.61s

KeyboardInterrupt: 

In [ ]:
with open(CURRENT_COHORT_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT_CURRENT = file.read()

In [ ]:
%%bigquery
$SQL_CONTENT_CURRENT

Executing query with job ID: 4bbcac1c-fbef-4fc5-b6a1-0f49ebf46edc
Query executing: 7.20s

KeyboardInterrupt: 

In [ ]:
%%bigquery
SELECT COUNT(*) FROM `dhh-ncr-stg.patrick_doupe.current_cohort_vendor_base`

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,4402892


In [ ]:
%%bigquery
DECLARE REPORT_DATE date DEFAULT '2025-06-01';
DECLARE target_entities ARRAY<STRING>; 
SET target_entities = ['FP_SG', 'TB_AE']; 

SELECT 
    COUNT(DISTINCT vendor_code)
FROM `fulfillment-dwh-production.cl_vendor.growth_vendor_smart_recommendations`
WHERE created_date = REPORT_DATE 
AND entity_id IN UNNEST(target_entities)

Query is running:   0%|          |

Downloading:   0%|          |

,f0_
0,50390


In [ ]:
%%bigquery
SELECT * FROM `dhh-ncr-stg.patrick_doupe.current_cohort_vendor_base`
LIMIT 10

Query is running:   0%|          |

Downloading:   0%|          |

,global_entity_id,vendor_id,cohort_id,created_month,gmv,customers,orders
0,TB_BH,661687,213,2024-07-01,2305.343000000,151,169
1,TB_KW,680757,485,2024-07-01,3107.040000000,176,260
2,TB_KW,637854,485,2024-07-01,0E-9,0,0
3,TB_KW,751957,653,2024-07-01,0E-9,0,0
4,TB_KW,602189,653,2024-07-01,3314.980000000,84,101
5,TB_AE,698537,874,2024-07-01,7076.220000000,398,525
6,TB_AE,758789,958,2024-07-01,0E-9,0,0
7,TB_AE,695952,1024,2024-07-01,9635.340000000,477,769
8,TB_KW,727825,1224,2024-07-01,2336.800000000,105,107
9,TB_KW,743582,1224,2024-07-01,0E-9,0,0


## Current recommendations

In [7]:
with open(CURRENT_RECOMMENDATIONS_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT = file.read()

In [8]:
%%bigquery
$SQL_CONTENT

Query is running:   0%|          |

""


In [5]:
with open(CURRENT_PERFORMANCE_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT = file.read()

In [6]:
%%bigquery
$SQL_CONTENT

Query is running:   0%|          |

""


In [9]:
with open(BEST_MATCH_RECOMMENDATIONS_SQL_FILE_PATH, 'r') as file:
    SQL_CONTENT = file.read()

In [10]:
%%bigquery
$SQL_CONTENT

Query is running:   0%|          |

""
